In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

import tensorflow as tf 

import yfinance as yf 

#    Fetching the stock data

Price and volume

In [4]:
ohlc = data.history(period="10y", interval="1d")  
volume = ohlc['Volume']

Latest quaterly financials 

In [ ]:
dividends = data.dividends
splits = data.splits
actions = data.actions
capital_gains = data.capital_gains

income_stmt = data.income_stmt
balance_sheet = data.balance_sheet
cashflow = data.cash_flow
tmm_cashflow = data.ttm_cash_flow
earnings = data.earnings

c:\Users\dawid\Desktop\BSEPrediction\.venv\Lib\site-packages\yfinance\scrapers\fundamentals.py:36: DeprecationWarning: 'Ticker.earnings' is deprecated as not available via API. Look for "Net Income" in Ticker.income_stmt.
  warnings.warn("'Ticker.earnings' is deprecated as not available via API. Look for \"Net Income\" in Ticker.income_stmt.", DeprecationWarning)


Additional data

In [6]:
info = data.info
fast_info = data.fast_info
calendar = data.calendar
earnings_dates = data.earnings_dates
# sec_fillings = data.sec_fillings

Analysis 

In [7]:
recommendations = data.recommendations
recommendations_summary = data.recommendations_summary

upgrades_downgrades = data.upgrades_downgrades
sustainability = data.sustainability

analyst_price_targets = data.analyst_price_targets
earnings_estimate = data.earnings_estimate
revenue_estimate = data.revenue_estimate

earnings_history = data.earnings_history
eps_trend = data.eps_trend
# eps_revision = data.eps_revision

Holdings

In [8]:
funds_data = data.funds_data
insider_purchases = data.insider_purchases
insider_transactions = data.insider_transactions
insider_roster_holders = data.insider_roster_holders
major_holders = data.major_holders
# institutional_holder = data.institutional_holder
mutualfund_holders = data.mutualfund_holders

# Fetching the market data

In [9]:
us_market = yf.Market('US')
gb_market = yf.Market('GB')

asian_market = yf.Market('ASIA')
european_market = yf.Market('EUROPE')

rates = yf.Market('RATES')
commodities = yf.Market('COMMODITIES')
currencies = yf.Market('CURRENCIES')
crypto = yf.Market('CRYPTOCURRENCIES')

# Sector and industry

In [10]:
print(yf.Sector.industries) # to be continued

# Data cleaning and preprocessing

In [12]:
total_dividend_through_years = dividends.sum()
total_number_of_stock_splits = len(splits)
# Process capital gains 
income_stmt

,2024-09-30,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0,NaN
Tax Rate For Calcs,0.241,0.147,0.162,0.133,NaN
Normalized EBITDA,134661000000.0,125820000000.0,130541000000.0,123136000000.0,NaN
Net Income From Continuing Operation Net Minority Interest,93736000000.0,96995000000.0,99803000000.0,94680000000.0,NaN
Reconciled Depreciation,11445000000.0,11519000000.0,11104000000.0,11284000000.0,NaN
Reconciled Cost Of Revenue,210352000000.0,214137000000.0,223546000000.0,212981000000.0,NaN
EBITDA,134661000000.0,125820000000.0,130541000000.0,123136000000.0,NaN
EBIT,123216000000.0,114301000000.0,119437000000.0,111852000000.0,NaN
Net Interest Income,NaN,-183000000.0,-106000000.0,198000000.0,890000000.0
Interest Expense,NaN,3933000000.0,2931000000.0,2645000000.0,2873000000.0


# Model creation

Add features like technical indicators

Random forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler